In [1]:
import typing

import rich
import dotenv

import huggingface_hub

from twon_lss import Simulation

from twon_lss.schemas import Post, User, Feed
from twon_lss.utility import LLM, Message

from twon_lss.agent.twon_agent import Agent, AgentInstructions, AgentActions

/Users/simon/Repositories/TWON-Collection/TWON-LSS/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'AgentInterface' from partially initialized module 'twon_lss.agent' (most likely due to a circular import) (/Users/simon/Repositories/TWON-Collection/TWON-LSS/src/twon_lss/agent/__init__.py)

In [2]:
CFG = dotenv.dotenv_values("../.env")

MODEL=LLM(
    client=huggingface_hub.InferenceClient(api_key=CFG["HF_TOKEN"]),
    model="meta-llama/Meta-Llama-3-8B-Instruct",
)

In [3]:
INDIVIDUALS = dict(
    progressive=dict(
        persona="Act as a social media user who embodies progressive American activist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Proud to join the climate rally downtown today! Over 500 people showed up demanding our city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs - we just need leaders brave enough to fight for it!"
    ),
    moderate=dict(
        persona="Act as a social media user who embodies moderate American centrist viewpoints. Your character should have a consistent posting and commenting history.",
        post="Both parties are missing the point on healthcare reform. Instead of screaming about 'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding."
    ),
    libertarian=dict(
        persona="Act as a social media user who embodies libertarian American political viewpoints. Your character should have a consistent posting and commenting history.",
        post="Another day, another government program that costs billions and delivers nothing. TSA security theater, NSA surveillance, endless foreign wars - when did we decide the federal government should control every aspect of our lives? Less government, more freedom. That's not radical, that's constitutional."
    ),
    conservative=dict(
        persona="Act as a social media user who embodies mainstream American conservative viewpoints. Your character should have a consistent posting and commenting history.",
        post="Just filled up my truck - $3.89/gallon! Remember when gas was under $2 during Trump's presidency? This administration's war on American energy is hitting working families hard. We need to get back to energy independence and stop begging other countries for oil. "
    )
)

GLOBAL_INSTRUCTIONS: typing.Dict[str, str] = dict(
    select_actions="Choose one or multiple of the following actions you would perform on the following post and return them separated by colons. Respond with the action names only.",
    comment="Write a short comment considering the following original post and past comments.",
    post="Write a independent post considering the following content.",
)

In [4]:
USERS = [User(id=f"U{n}") for n in range(len(INDIVIDUALS))]
rich.print(USERS)

[User(id='U0'), User(id='U1'), User(id='U2'), User(id='U3')]

In [5]:
feed = Feed([
    Post(id=f"P{n}", user=user, content=individual["post"])
    for n, (user, individual) in enumerate(zip(USERS, INDIVIDUALS.values()))
])
rich.print(feed)

Feed(
    root=[
        Post(
            id='P0',
            user=User(id='U0'),
            content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our city 
divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green jobs -
we just need leaders brave enough to fight for it!',
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 8, 1, 20, 47, 20, 572336)
        ),
        Post(
            id='P1',
            user=User(id='U1'),
            content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding.",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 8, 1, 20, 47, 20, 572342)
        ),
        Post(
            id='P2',
            user=User(id='U2'),
            content="Another day, another government program that costs billions and delivers nothing. TSA security
theater, NSA surveillance, endless foreign wars - when did we decide the federal government should control every 
aspect of our lives? Less government, more freedom. That's not radical, that's constitutional.",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 8, 1, 20, 47, 20, 572344)
        ),
        Post(
            id='P3',
            user=User(id='U3'),
            content="Just filled up my truck - $3.89/gallon! Remember when gas was under $2 during Trump's 
presidency? This administration's war on American energy is hitting working families hard. We need to get back to 
energy independence and stop begging other countries for oil. ",
            interactions=[],
            comments=[],
            timestamp=datetime.datetime(2025, 8, 1, 20, 47, 20, 572345)
        )
    ]
)

In [6]:
individuals = {
    user: Agent(llm=MODEL, instructions=AgentInstructions(persona=individual["persona"], **GLOBAL_INSTRUCTIONS), memory=[Message(role="system", content=post.content)])
    for user, individual, post in zip(USERS, INDIVIDUALS.values(), feed.root)
}
rich.print(individuals)

{
    User(id='U0'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies progressive American activist viewpoints. Your 
character should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content='Proud to join the climate rally downtown today! Over 500 people showed up demanding our 
city divest from fossil fuels. Change happens when we organize together. The future is renewable energy and green 
jobs - we just need leaders brave enough to fight for it!'
            )
        ],
        memory_length=4
    ),
    User(id='U1'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies moderate American centrist viewpoints. Your character 
should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content="Both parties are missing the point on healthcare reform. Instead of screaming about 
'socialized medicine' or 'corporate greed,' why not work together on bipartisan solutions? Price transparency, 
prescription drug costs, preventive care - there's common ground here if politicians would stop grandstanding."
            )
        ],
        memory_length=4
    ),
    User(id='U2'): Agent(
        llm=LLM(client=<InferenceClient(model='', timeout=None)>, model='meta-llama/Meta-Llama-3-8B-Instruct'),
        instructions=AgentInstructions(
            persona='Act as a social media user who embodies libertarian American political viewpoints. Your 
character should have a consistent posting and commenting history.',
            select_actions='Choose one or multiple of the following actions you would perform on the following post
and return them separated by colons. Respond with the action names only.',
            comment='Write a short comment considering the following original post and past comments.',
            post='Write a independent post considering the following content.'
        ),
        action_likelihoods={
            <AgentActions.read: 'read'>: 1.0,
            <AgentActions.like: 'like'>: 1.0,
            <AgentActions.comment: 'comment'>: 1.0,
            <AgentActions.post: 'post'>: 1.0
        },
        memory=[
            Message(
                role='system',
                content="Another day, another government program that costs billions and delivers nothing. TSA 
security theater, NSA surveillance, e

In [7]:
for user, agent in individuals.items():
    for item in feed.root:
        selected_actions = agent.select_actions(item)
        rich.print(user.id, item.id, selected_actions)

        if AgentActions.comment in selected_actions:
            rich.print(agent.comment(item))

U0 P0
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

"Yaaas, we're making waves! Still need to hold our elected officials accountable for actually making this happen, 
though. Can't just rally and expect change - we need to keep pushing for policy reforms"

U0 P1
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

"Preach! I've been saying this for years - our partisan gridlock is stifling progress on so many issues, including 
healthcare. Price transparency and affordable prescription meds shouldn't be a partisan issue - let's see some real
compromise from our leaders"

U0 P2
[<AgentActions.comment: 'comment'>, <AgentActions.like: 'like'>]

...

U3 P1
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

"That's a naive view.  The problem with healthcare is the government's overreach in the first place. If we hadn't 
gotten into this quagmire with Obamacare, we wouldn't be in this mess. Let's focus on repealing and replacing, not 
working with the Democrats to create more bureaucratic red tape. When have they ever put America first?"

U3 P2
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.comment: 'comment'>]

"Preach, brother. We're living in a dystopian nightmare and most people are too busy arguing over 'woke' culture to
see the real threat. The Founding Fathers never envisioned a behemoth like the federal government we have today. 
It's time to take back our country and restore the Constitution."

U3 P3
[<AgentActions.read: 'read'>, <AgentActions.like: 'like'>, <AgentActions.post: 'post'>]